In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import importlib
import sys
import nrrd
import numpy as np

sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets')
from dl.nets import us_simu

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
diffusor_np, diffusor_head = nrrd.read('/mnt/raid//C1_ML_Analysis/simulated_data_export/placenta/FAM-025-0664-4_label11_resampled.nrrd')
diffusor_t = torch.tensor(diffusor_np.astype(int)).permute(2, 1, 0).cuda()
print(diffusor_head)
diffusor_size = torch.tensor(diffusor_head['sizes']).cuda()
diffusor_spacing = torch.tensor(np.diag(diffusor_head['space directions'])).cuda()

diffusor_origin = torch.tensor(diffusor_head['space origin']).flip(dims=[0]).cuda()
diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size


In [ ]:
model = us_simu.USPCReconstruction(mount_point='/mnt/raid/C1_ML_Analysis', 
    latent_channels=3,
    num_samples=4096,
    input_dim=6,
    output_dim=3,
    K=[27, 27],
    num_heads=[64, 128],
    stages=[64, 128],
    dropout=0.1,
    pooling_factor=[0.5, 0.5],
    score_pooling=0,
    pooling_hidden_dim=[32, 64],
    feed_forward_hidden_dim=[32, 64],
    use_skip_connection=0, 
    num_random_sweeps=3,
    n_grids=200, 
    target_label=7, 
    n_fixed_samples=12288).cuda()

In [4]:
# x_sweeps, x_sweeps_tags = model.volume_sampling(diffusor_t.unsqueeze(0).unsqueeze(0).repeat([2,1,1,1,1]), diffusor_origin.unsqueeze(0).repeat([2,1,1,1]), diffusor_end.unsqueeze(0).repeat([2,1,1,1]))    
# print(x_sweeps.shape, x_sweeps_tags.shape)
# print(x_sweeps_tags[0])

# fig = px.imshow(diffusor_t.squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [5]:
# fig = px.imshow(x_sweeps.cpu().numpy()[0][0][0], animation_frame=0, binary_string=True)
# fig.show()

In [ ]:
model.on_fit_start()
model.training_step((diffusor_t.unsqueeze(0).unsqueeze(0).repeat([2,1,1,1,1]), diffusor_origin.unsqueeze(0).repeat([2,1,1,1]), diffusor_end.unsqueeze(0).repeat([2,1,1,1])), 0)

In [7]:
y_v = model.get_target(diffusor_t.unsqueeze(0).unsqueeze(0).repeat([2,1,1,1,1]), diffusor_origin.unsqueeze(0).repeat([2,1,1,1]), diffusor_end.unsqueeze(0).repeat([2,1,1,1]))


In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=y_v[0,:,0].cpu().numpy(), y=y_v[0,:,1].cpu().numpy(), z=y_v[0,:,2].cpu().numpy(), mode='markers', marker=dict(
        size=2,
        # color=y_v[0,:, 2].cpu().numpy().squeeze(),                # set color to an array/list of desired values
        colorscale='jet',   # choose a colorscale
        opacity=0.5
    ))])
fig.show()

In [ ]:
y_v.shape